In [1]:
from mqt.bench import CompilerSettings, QiskitSettings, TKETSettings, get_benchmark
from qiskit import *
import numpy as np
from qiskit_aer import AerSimulator
from qiskit import *
from qiskit.providers.fake_provider import *
import numpy as np 
import os

In [2]:
import pandas as pd
import time
import csv


from qiskit.primitives import StatevectorSampler
sampler = StatevectorSampler()

In [3]:
all_algo = ["ae","dj","ghz","graphstate","qaoa","portfolioqaoa","qft","vqe","portfoliovqe","qftentangled","qpeexact","qpeinexact","random","realamprandom","twolocalrandom","qwalk-noancilla","wstate","grover-v-chain","qwalk-v-chain","grover-noancilla","qnn","pricingcall","pricingput","routing","tsp"]
#all_algo = ["ae","dj"]

In [ ]:
records = []
csv_file = "benchmark_results_all_mqt_alogs_4_to_18.csv"

compiler_settings = CompilerSettings(qiskit=QiskitSettings(optimization_level=1))
for i in all_algo:
    
    for n in range(4, 18, 2):
        
        qc = get_benchmark(benchmark_name=i,
                           level="nativegates",
                           circuit_size=n,
                           compiler="qiskit",
                           compiler_settings=compiler_settings,
                           provider_name="ionq",)
        #print("---------------",i,n,"---------------")
        #print(qc_native_gates_level.draw())

        
        start =time.time()
        job = sampler.run([qc], shots=1000)
        end =time.time()
        result = job.result()

        
        #counts = result[0].data["meas"].get_counts()
        #print(f" > Counts: {result[0].data["meas"].get_counts()}")
        
        print("Algorithm: ",i,"with",n,"qubit","is taking =",end-start, "seconds for execution.")
        records.append([i, n, end-start])
        file_is_empty = (not os.path.exists(csv_file)) or (os.path.getsize(csv_file) == 0)

        with open(csv_file, "a", newline="") as f:
            writer = csv.writer(f)
            if file_is_empty:
                writer.writerow(["Algorithm", "Qubit number", "Time taken in Seconds"])
            writer.writerow([i, n, end-start])
print("CSV updated:", csv_file)

Algorithm:  ae with 4 qubit is taking = 0.005810260772705078 seconds for execution.
Algorithm:  ae with 6 qubit is taking = 0.005552768707275391 seconds for execution.
Algorithm:  ae with 8 qubit is taking = 0.0055980682373046875 seconds for execution.
Algorithm:  ae with 10 qubit is taking = 0.0056896209716796875 seconds for execution.
Algorithm:  ae with 12 qubit is taking = 0.005629062652587891 seconds for execution.
Algorithm:  ae with 14 qubit is taking = 0.005500078201293945 seconds for execution.
Algorithm:  ae with 16 qubit is taking = 0.0056340694427490234 seconds for execution.
Algorithm:  dj with 4 qubit is taking = 0.0025789737701416016 seconds for execution.
Algorithm:  dj with 6 qubit is taking = 0.0024847984313964844 seconds for execution.
Algorithm:  dj with 8 qubit is taking = 0.0032699108123779297 seconds for execution.
Algorithm:  dj with 10 qubit is taking = 0.0035779476165771484 seconds for execution.
Algorithm:  dj with 12 qubit is taking = 0.0034308433532714844 s

In [ ]:
df = pd.read_csv("benchmark_results_all_mqt_alogs_4_to_18.csv")
df

#df.style.set_properties(**{'text-align': 'left'})

In [ ]:
df = pd.DataFrame(records, columns=["Algorithm", "Qubits", "Time_Seconds"])
df.to_csv("benchmark_results.csv", index=False)

df.head()

import matplotlib.pyplot as plt

# Convert records to algorithm-wise mapping
algo_dict = {}

for algo, qubits, time_taken in records:
    if algo not in algo_dict:
        algo_dict[algo] = {"qubits": [], "times": []}
    algo_dict[algo]["qubits"].append(qubits)
    algo_dict[algo]["times"].append(time_taken)

# Plot one graph per algorithm
for algo in algo_dict:
    plt.figure(figsize=(8, 4))
    plt.plot(algo_dict[algo]["qubits"], algo_dict[algo]["times"], marker="o")
    plt.xlabel("Qubits")
    plt.ylabel("Time (seconds)")
    plt.title(f"Execution Time vs Qubits — {algo}")
    plt.grid(True)
    plt.show()

